# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 23 2022 3:02PM,247191,10,SNAP31975723,"Snap Medical Industries, LLC",Released
1,Sep 23 2022 3:02PM,247191,10,SNAP923221,"Snap Medical Industries, LLC",Executing
2,Sep 23 2022 3:02PM,247191,10,SNAP923222,"Snap Medical Industries, LLC",Released
3,Sep 23 2022 3:02PM,247191,10,SNAP923223,"Snap Medical Industries, LLC",Released
4,Sep 23 2022 3:02PM,247191,10,SNAP923224,"Snap Medical Industries, LLC",Released
5,Sep 23 2022 2:57PM,247189,10,8563427,MedStone Pharma LLC,Released
6,Sep 23 2022 2:57PM,247189,10,8563428,MedStone Pharma LLC,Released
7,Sep 23 2022 2:57PM,247189,10,8563429,MedStone Pharma LLC,Released
8,Sep 23 2022 2:57PM,247189,10,8563430,MedStone Pharma LLC,Released
9,Sep 23 2022 2:57PM,247189,10,8563431,MedStone Pharma LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,247188,Released,1
32,247189,Released,13
33,247190,Released,3
34,247191,Executing,1
35,247191,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247185,NaN,NaN,2.0
247188,NaN,3.0,1.0
247189,NaN,NaN,13.0
247190,NaN,NaN,3.0
247191,NaN,1.0,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247116,0.0,1.0,0.0
247129,0.0,0.0,1.0
247137,5.0,0.0,4.0
247138,0.0,0.0,1.0
247139,5.0,3.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247116,0,1,0
247129,0,0,1
247137,5,0,4
247138,0,0,1
247139,5,3,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247116,0,1,0
1,247129,0,0,1
2,247137,5,0,4
3,247138,0,0,1
4,247139,5,3,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247116,,1,
1,247129,,,1
2,247137,5,,4
3,247138,,,1
4,247139,5,3,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 23 2022 3:02PM,247191,10,"Snap Medical Industries, LLC"
5,Sep 23 2022 2:57PM,247189,10,MedStone Pharma LLC
18,Sep 23 2022 2:56PM,247190,10,ISDIN Corporation
21,Sep 23 2022 2:56PM,247182,21,"NBTY Global, Inc."
22,Sep 23 2022 2:55PM,247183,10,"Methapharm, Inc."
42,Sep 23 2022 2:55PM,247185,10,"Methapharm, Inc."
44,Sep 23 2022 2:55PM,247188,10,Emerginnova
48,Sep 23 2022 2:51PM,247178,21,"NBTY Global, Inc."
49,Sep 23 2022 2:50PM,247181,19,"AdvaGen Pharma, Ltd"
50,Sep 23 2022 2:49PM,247180,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 23 2022 3:02PM,247191,10,"Snap Medical Industries, LLC",,1,4
1,Sep 23 2022 2:57PM,247189,10,MedStone Pharma LLC,,,13
2,Sep 23 2022 2:56PM,247190,10,ISDIN Corporation,,,3
3,Sep 23 2022 2:56PM,247182,21,"NBTY Global, Inc.",,,1
4,Sep 23 2022 2:55PM,247183,10,"Methapharm, Inc.",,,20
5,Sep 23 2022 2:55PM,247185,10,"Methapharm, Inc.",,,2
6,Sep 23 2022 2:55PM,247188,10,Emerginnova,,3,1
7,Sep 23 2022 2:51PM,247178,21,"NBTY Global, Inc.",,1,
8,Sep 23 2022 2:50PM,247181,19,"AdvaGen Pharma, Ltd",,,1
9,Sep 23 2022 2:49PM,247180,10,ISDIN Corporation,,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 3:02PM,247191,10,"Snap Medical Industries, LLC",4,1,
1,Sep 23 2022 2:57PM,247189,10,MedStone Pharma LLC,13,,
2,Sep 23 2022 2:56PM,247190,10,ISDIN Corporation,3,,
3,Sep 23 2022 2:56PM,247182,21,"NBTY Global, Inc.",1,,
4,Sep 23 2022 2:55PM,247183,10,"Methapharm, Inc.",20,,
5,Sep 23 2022 2:55PM,247185,10,"Methapharm, Inc.",2,,
6,Sep 23 2022 2:55PM,247188,10,Emerginnova,1,3,
7,Sep 23 2022 2:51PM,247178,21,"NBTY Global, Inc.",,1,
8,Sep 23 2022 2:50PM,247181,19,"AdvaGen Pharma, Ltd",1,,
9,Sep 23 2022 2:49PM,247180,10,ISDIN Corporation,5,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 3:02PM,247191,10,"Snap Medical Industries, LLC",4,1,
1,Sep 23 2022 2:57PM,247189,10,MedStone Pharma LLC,13,,
2,Sep 23 2022 2:56PM,247190,10,ISDIN Corporation,3,,
3,Sep 23 2022 2:56PM,247182,21,"NBTY Global, Inc.",1,,
4,Sep 23 2022 2:55PM,247183,10,"Methapharm, Inc.",20,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 3:02PM,247191,10,"Snap Medical Industries, LLC",4.0,1.0,NaN
1,Sep 23 2022 2:57PM,247189,10,MedStone Pharma LLC,13.0,NaN,NaN
2,Sep 23 2022 2:56PM,247190,10,ISDIN Corporation,3.0,NaN,NaN
3,Sep 23 2022 2:56PM,247182,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Sep 23 2022 2:55PM,247183,10,"Methapharm, Inc.",20.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 3:02PM,247191,10,"Snap Medical Industries, LLC",4.0,1.0,0.0
1,Sep 23 2022 2:57PM,247189,10,MedStone Pharma LLC,13.0,0.0,0.0
2,Sep 23 2022 2:56PM,247190,10,ISDIN Corporation,3.0,0.0,0.0
3,Sep 23 2022 2:56PM,247182,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Sep 23 2022 2:55PM,247183,10,"Methapharm, Inc.",20.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4449003,107.0,10.0,15.0
16,494324,4.0,1.0,0.0
18,247175,1.0,0.0,0.0
19,247181,1.0,0.0,0.0
20,494339,2.0,1.0,2.0
21,741476,1.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4449003,107.0,10.0,15.0
1,16,494324,4.0,1.0,0.0
2,18,247175,1.0,0.0,0.0
3,19,247181,1.0,0.0,0.0
4,20,494339,2.0,1.0,2.0
5,21,741476,1.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,107.0,10.0,15.0
1,16,4.0,1.0,0.0
2,18,1.0,0.0,0.0
3,19,1.0,0.0,0.0
4,20,2.0,1.0,2.0
5,21,1.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,107.0
1,16,Released,4.0
2,18,Released,1.0
3,19,Released,1.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,18,19,20,21
Status,,,,,,
Completed,15.0,0.0,0.0,0.0,2.0,0.0
Executing,10.0,1.0,0.0,0.0,1.0,2.0
Released,107.0,4.0,1.0,1.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,18,19,20,21
0,Completed,15.0,0.0,0.0,0.0,2.0,0.0
1,Executing,10.0,1.0,0.0,0.0,1.0,2.0
2,Released,107.0,4.0,1.0,1.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,18,19,20,21
0,Completed,15.0,0.0,0.0,0.0,2.0,0.0
1,Executing,10.0,1.0,0.0,0.0,1.0,2.0
2,Released,107.0,4.0,1.0,1.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()